<a href="https://colab.research.google.com/github/h5ng/GNN/blob/master/cmod_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fairseq==0.9 transformers==2.9

In [2]:
import csv
import os
import logging
import argparse
import random
from tqdm import tqdm, trange
import json

import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers.tokenization_bert import BertTokenizer
from transformers.modeling_bert import BertForMaskedLM, BertOnlyMLMHead

from transformers import AdamW

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/transformers-data-augmentation/bert_aug')
from data_processors import get_task_processor

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BERT_MODEL = 'bert-base-uncased'

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

In [4]:
from argparse import Namespace
parser = argparse.ArgumentParser()
args = {
    'data_dir': '/content/drive/MyDrive/transformers-data-augmentation/datasets/TREC',
    'output_dir': 'aug_data',
    'task_name': 'trec',
    'max_seq_length': 64,
    'cache': 'transformers_cache',
    'train_batch_size': 8,
    'learning_rate': 4e-5,
    'num_train_epochs': 10.0,
    'warmup_proportion': 0.1,
    'seed': 42,
    'sample_num': 1,
    'sample_ratio': 7,
    'gpu': 0,
    'temp': 1.0
}
args = Namespace(**args)

In [24]:
def train_cmodbert_and_augment(args):
  print(args)
  task_name = args.task_name
  os.makedirs(args.output_dir, exist_ok=True)

  random.seed(args.seed)
  np.random.seed(args.seed)
  torch.manual_seed(args.seed)

  processor = get_task_processor(task_name, args.data_dir)
  label_list = processor.get_labels(task_name)

  # load train and dev data
  train_examples = processor.get_train_examples()
  dev_examples = processor.get_dev_examples()

  tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=True, cache_dir=args.cache)


In [25]:
train_cmodbert_and_augment(args)

Namespace(cache='transformers_cache', data_dir='/content/drive/MyDrive/transformers-data-augmentation/datasets/TREC', gpu=0, learning_rate=4e-05, max_seq_length=64, num_train_epochs=10.0, output_dir='aug_data', sample_num=1, sample_ratio=7, seed=42, task_name='trec', temp=1.0, train_batch_size=8, warmup_proportion=0.1)
/content/drive/MyDrive/transformers-data-augmentation/datasets/TREC


NameError: ignored